##Model Detailing

In this task i have assumed the close value as the analysis parameter to obtain the maximum profit returning share for future.I had tried to apply the "percent stock return for next week" as the parameter to obtain the profit but unfortunately i was getting a pretty bad accuracy of the model. As there are 30 companies we calculate the percent profit return of each of the companies based on the given 25 record values of each stocks.After obtaining the stock profit return for each stock we proceed with by filtering the highest profit returning stock out of 30 given stocks.

## 1. Import the Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from numpy import array

df=pd.read_csv('https://code.datasciencedojo.com/datasciencedojo/datasets/raw/master/Dow%20Jones%20Index/dow_jones_index.data',sep=',')

## 2. Create a list to store profit percentage values of all the stocks

In [ ]:
profit_all_stocks=[]

## 3. Find the unique stocks present in the dataset

In [ ]:
stocks_names=list(df['stock'].unique())
print(stocks_names)

['AA', 'AXP', 'BA', 'BAC', 'CAT', 'CSCO', 'CVX', 'DD', 'DIS', 'GE', 'HD', 'HPQ', 'IBM', 'INTC', 'JNJ', 'JPM', 'KRFT', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'PFE', 'PG', 'T', 'TRV', 'UTX', 'VZ', 'WMT', 'XOM']


## 4. Create a method that returns the percent of stock returns of each shares

We initally divide the total records of each company into training and test dataset.However we need to note that the data given is sequential data i.e. the stock values in close column is weekly present in the dataset. Our goal is to make sure that the data arrangement into train and test set is not split randomly but its done sequentially. So if we want to train a model that predicts x week's data then we need to make sure that the input to the model is x-n weeks' data where n represents how many past weeks must be given as input.










Next we create a model by training and testing using regression model. Once this is done we try to forecast the next 'm' no. of weeks' stock values using the already trained model.



Once we get the forecasted values of next 5 weeks we then take the difference between all the 5 weeks close values and calculate the percentage different in return in terms of each week.


Once we have the return percentage for each of the weeks we calculate the average value of return percentage over given 5 weeks and calculate the return percentage for the share which is obtained using the return of the below function

In [ ]:
def get_stock_return(dataframe,stock_name):
  Stock_data=dataframe[dataframe['stock']==stock_name]
  

  #<----Remove the unwanted string in Close stock price values---->#
  list2=[]
  list1=list(Stock_data.loc[:,'close'])
  for item in list1:
    item=item.replace('$','')
    list2.append(item)
  Stock_data['close']=list2
  





  #<----Convert from string to float values in Close column and get only the close column to perform future analysis---->#
  Stock_data['close'] = pd.to_numeric(Stock_data['close'], downcast="float")
  Stock_data=Stock_data.reset_index()['close']
  



  #<----Convert into Scaler Form---->#
  scaler=StandardScaler()
  Stock_data=scaler.fit_transform(np.array(Stock_data).reshape(-1,1))




  #<----Split into training and test data.---->#
  #<----As it is time based sequence first 65% will be put into train dataset and rest into test dataset---->#
  training_size=int(len(Stock_data)*0.65)
  test_size=len(Stock_data)-training_size
  train_data,test_data=Stock_data[0:training_size,:],Stock_data[training_size:len(Stock_data),:1]



  #<----Call the create_dataset to arrange the data based on time_step---->#
  #<----here we set time_Step to 2 which means the output depends on 2 previous values of stock price---->#
  time_step = 2
  X_train, y_train = create_dataset(train_data, time_step)
  X_test, ytest = create_dataset(test_data, time_step)




  #<----Apply the regression model on train and test data---->#
  regression = LinearRegression()
  regression.fit(X_train, y_train)
  predicted=regression.predict(X_test)




  #<----Convert the test data back to inverse Scaler mode to compare the original values---->#
  predicted=predicted.reshape(-1,1)
  ytest=ytest.reshape(-1,1)
  predicted = scaler.inverse_transform(predicted)
  ytest = scaler.inverse_transform(ytest)
  ytest = ytest.flatten()
  predicted=predicted.flatten()




  #<----Create a dataframe to compare the actual and expected value---->#
  dfr=pd.DataFrame({'Actual_Price':ytest, 'Predicted_Price':predicted})
  #print(dfr)




  #<----Comparison Matrix---->#
  x2 = dfr.Actual_Price.mean()
  y2 = dfr.Predicted_Price.mean()
  Accuracy1 = x2/y2*100
  #print("The accuracy of the model is " , Accuracy1)




  #<----Create a temp input with last 2 items from the test data---->#
  x_input=test_data[6:].reshape(1,-1)
  temp_input=list(x_input)
  temp_input=temp_input[0].tolist()

  
  #<----Extract the last week close value from given data for future calculation---->#
  x_input = scaler.inverse_transform(x_input)
  lst_input=x_input[0][1]


  #<----Predict the future 5 weeks values for the stock---->#
  lst_output=future_forecast(temp_input,x_input,regression)



  #<----Convert the future values into original values using Inverse transform---->#
  lst_output = scaler.inverse_transform(lst_output)
  lst_output = lst_output.tolist()
  lst_output.insert(0,lst_input)
  #print(lst_output)
  

  #<----Calculate the profit returns weekly difference---->#
  prev_week=lst_output[0]
  lst_percnt_weekly=[]
  for item in lst_output[1:]:
    percent=((item-prev_week)/prev_week)*100
    lst_percnt_weekly.append(percent)
    prev_week=item
  
  #<----Overal profit returns over 5 weeks---->#
  sum=0
  for item in lst_percnt_weekly:
    sum=sum+item
  stock_profit=(sum/5)

  profit_all_stocks.append(stock_profit)
  return stock_profit

## 5. Create the training and testing dataset using the below method

In [ ]:
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return np.array(dataX), np.array(dataY)

## 6. The forecasting method to predict stock values over 5 weeks and find the percentage return over 5 weeks

In [ ]:
def future_forecast(temp_input,x_input,regression):
  lst_output=[]
  n_steps=2
  i=0
  j=1
  while(i<5):
    if(len(temp_input)>2):
      #print(temp_input)
      x_input=np.array(temp_input[j:])
      #print("{} week input {}".format(i,x_input))
      x_input=x_input.reshape(1,-1)
      yhat = regression.predict(x_input)
      #print("{} week output {}".format(i,yhat))
      temp_input.extend(yhat.tolist())
      temp_input=temp_input[:]
      lst_output.extend(yhat.tolist())
      j=j+1
      i=i+1
    else:
      yhat = regression.predict(x_input)
      temp_input.append(yhat[0])
      lst_output.extend(yhat.tolist())
      i=i+1

  return lst_output

## 7. Calculate the total stock returns for each company

In [ ]:
dict={}
for item in stocks_names:
  stock_return=get_stock_return(df,item)
  dict[item]=stock_return
  print("Total stock return for ",item,"is given by ",stock_return,"%.")

Total stock return for  AA is given by  1.7329137844167555 %.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

Total stock return for  AXP is given by  -1.1696588734333617 %.
Total stock return for  BA is given by  -0.5698321697785326 %.
Total stock return for  BAC is given by  5.138575171441034 %.
Total stock return for  CAT is given by  1.874238451183717 %.
Total stock return for  CSCO is given by  1.0546020233643736 %.
Total stock return for  CVX is given by  0.8517411597678051 %.
Total stock return for  DD is given by  1.7967424815572877 %.
Total stock return for  DIS is given by  2.0761037405664173 %.
Total stock return for  GE is given by  2.092087082991144 %.
Total stock return for  HD is given by  1.587529314072071 %.
Total stock return for  HPQ is given by  2.2755327148774693 %.
Total stock return for  IBM is given by  -0.14772120897270338 %.
Total stock return for  INTC is given by  -0.45842614146505 %.
Total stock return for  JNJ is given by  -1.8040541211509762 %.
Total stock return for  JPM is given by  2.284448187444655 %.
Total stock return for  KRFT is given by  -0.1090458072551

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

## 8. Mapping of Company with the Stock Return values

In [ ]:
print(dict)

{'AA': 1.7329137844167555, 'AXP': -1.1696588734333617, 'BA': -0.5698321697785326, 'BAC': 5.138575171441034, 'CAT': 1.874238451183717, 'CSCO': 1.0546020233643736, 'CVX': 0.8517411597678051, 'DD': 1.7967424815572877, 'DIS': 2.0761037405664173, 'GE': 2.092087082991144, 'HD': 1.587529314072071, 'HPQ': 2.2755327148774693, 'IBM': -0.14772120897270338, 'INTC': -0.45842614146505, 'JNJ': -1.8040541211509762, 'JPM': 2.284448187444655, 'KRFT': -0.10904580725510508, 'KO': 0.13389087734097943, 'MCD': -1.726960761191735, 'MMM': -0.06262143398343277, 'MRK': -1.2428444216989627, 'MSFT': 0.20779255197092775, 'PFE': 0.6765052602242652, 'PG': -0.5589273270469718, 'T': -0.9694145082855616, 'TRV': 0.3593643836974616, 'UTX': -0.42496202980863024, 'VZ': 0.7483717991221301, 'WMT': 0.17458860140825797, 'XOM': 1.0632152478274557}


## 9. Find the maxmimum stock value 

In [ ]:
max_return_value=max(profit_all_stocks)

## 10. Find the stock that can return highest return over 5 weeks of time

In [ ]:
keys = [k for k, v in dict.items() if v == max_return_value]
print("The stock",keys," can give maximum return over 5 weeks of time with ",max_return_value,"%")

The stock ['BAC']  can give maximum return over 5 weeks of time with  5.138575171441034 %
